In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import xmltodict
import os
import json
import pickle
from lxml import etree


In [3]:
# mode = int(metadata['mode']) if metadata['mode'] is not None else 1
# beats_in_measure = int(metadata['beats_in_measure'])

# melody, chord = segments_parser(segments, mode, beats_in_measure)

In [4]:
from pathlib import Path

In [5]:
version = 'v7'
data_path = Path('data/midi')
version_path = data_path/version
orig_path = version_path/'midi_sources'

In [6]:
from fastai.data_block import get_files

In [7]:
h_path = orig_path/'hooktheory'

In [8]:
files = get_files(h_path, extensions=['.xml'], recurse=True); files[:10]

[PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wayne-sharpe/yu-gi-oh-theme-song/chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wayne-sharpe/yu-gi-oh-theme-song/intro.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/what-a-day/kiefer/chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/whiteflame/senbonzakura/pre-chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/whiteflame/senbonzakura/verse.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/whiteflame/senbonzakura/chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wham/last-christmas/verse.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wham/last-christmas/chorus.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wham/last-christmas/intro.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/w/wham/freedom/chorus.xml')]

In [9]:
# Loading from specific file
# keywords = ['get-lucky', 'daft-punk', 'pre-chorus']
# keywords = ['skrillex', 'scary']
keywords = ['idina', 'verse', 'let']
# keywords = ['game-of-thrones', 'intro', 'ramin']
# keywords = ['kiss-from-a-rose', 'seal']
def contains_keywords(f): return all([k in str(f) for k in keywords])
search = [f for f in files if contains_keywords(f)]; search

[PosixPath('data/midi/v7/midi_sources/hooktheory/xml/i/idina-menzel/let-it-go/verse.xml'),
 PosixPath('data/midi/v7/midi_sources/hooktheory/xml/i/idina-menzel/let-it-go/intro-and-verse.xml')]

In [10]:
from src.tab_parser import *

In [12]:
from src import roman_to_symbol
from src import to_pianoroll
from collections import defaultdict
from midi_data import keyc_offset

In [13]:
def parse_file(file_path):
    content = load_data(file_path)
    root = xml_parser(content)
    metadata, version = get_metadata(root)
    segments, num_measures = get_lead_sheet(root, version)
    
    song = HSong.parse(metadata, segments)
    return song

In [14]:
import music21

### Create config file

In [15]:
config_opts = dict(sustain=True, sep_octave=True, note_octave=4, chord_octave=3, 
              ts='4/4', ks=0, bpm=120, freq=2, pad_idx=-1, none_idx=0, sus_idx=2, hit_idx=1)

class Config(object):
    def __init__(self, d): self.__dict__ = d
        
config = Config(config_opts)

### Constants

In [16]:
MODE_TO_KEYOFFSET = {
    '1': 0,
    '2': 2,
    '3': 4,
    '4': 5,
    '5': 7,
    '6': 9,
    '7': 11
#     '5': -5,
#     '6': -3,
#     '7': -1
}

In [17]:
PITCH_TO_SD = {
    0: '1',
    1: '1#',
    2: '2',
    3: '2#',
    4: '3',
    5: '4',
    6: '4#',
    7: '5',
    8: '5#',
    9: '6',
    10:'6#',
    11:'7',
}

SD_TO_PITCH = {v:k for k,v in PITCH_TO_SD.items()}

### Classes

In [18]:
from dataclasses import dataclass
import dataclasses
from typing import Dict, Any, AnyStr, List, Sequence, TypeVar, Tuple, Optional, Union

In [19]:
def parse(cls, d):
    cls_keys = cls.__dataclass_fields__.keys()
    kwargs = {key:d[key] for key in cls_keys}
    return cls(**kwargs)

@dataclass
class Base:
    @classmethod
    def from_dict(cls, d):
        cls_keys = cls.__dataclass_fields__.keys()
        kwargs = {key:d[key] for key in cls_keys}
        return cls(**kwargs)
    
    @classmethod
    def parse(cls, d):
        return cls.from_dict(d)

In [20]:
@dataclass
class HMetadata(Base):
    title:str
    BPM:str='120'
    beats_in_measure:str='4'
    key:str='C'
    mode:str='1'

In [21]:
@dataclass
class HNote(Base):
    beat_abs:float
    measure:float
    beat:float
    duration:float
    scale:str
    octave: str
        
    def to_m21(self)->music21.note.Note:
#         if self.scale_degree == 'rest': return None, None
#             n = music21.note.Rest(quarterLength=note_length)
        pitch = self.pitch() + 12*(int(self.octave)+config.note_octave)
        n = music21.note.Note(pitch, quarterLength=self.duration)
#         key.KeySignature(-1)
        return n, float(self.beat_abs)
    
    def pitch(self):
        return SD_TO_PITCH[self.scale]
    
    def end_time(self):
        return self.duration + self.beat_abs
    
    @classmethod
    def parse(cls, d, mode, key_offset):
#         if key_offset > 5: key_offset = key_offset-12
        parsed = roman_to_symbol.hnote_parser(d, mode, key_offset)
        pitch = parsed['pitch']
        scale_degree = PITCH_TO_SD[int((pitch) % 12)]
        octave = (pitch // 12) + 1
        m = {
            'scale': scale_degree,
            'octave': octave,
            'duration': float(d['note_length']),
            'measure': float(d['start_measure']),
            'beat': float(d['start_beat']),
            'beat_abs': float(d['start_beat_abs']),
        }
        return cls.from_dict({**d, **m})

In [167]:
def last_comp(comp):
    if comp is None: return config.none_idx
    return int(comp[-1])



@dataclass
class HChord(Base):
    # ht relative
    scale:int
    base:int
    sus:int
    
    # ht tempo
    duration:float # (AS) TODO: convert to float
    measure:float
    beat:float
    beat_abs:float # (AS) TODO: convert to float
        
    # abs
    composition:List[int]
    symbol:str=None
    quality:str=None
        
    def end_time(self):
        return self.duration + self.beat_abs
        
    def to_m21(self)->music21.chord.Chord:
        notes = [n+config.chord_octave*12 for n in self.composition]
        c = music21.chord.Chord(notes, quarterLength=self.duration)
        return c, float(self.beat_abs)

    @classmethod
    def parse(cls, d, mode, key_offset, reset_to_base=True):
        parsed = roman_to_symbol.hchord_parser(d, mode, 0)
        
        # After offset, let's reset the chord to be the lowest possible offset on new scale
#         if reset_to_base:
#             lowest = min(parsed['composition'])
#             if lowest > 12:
#                 parsed = roman_to_symbol.chord_key_shifting(parsed, -12)
#                 # reset chord name
#                 new_s = roman_to_symbol.chord_to_string(parsed)
#                 parsed['symbol'] = new_s
        
        parsed['composition'] = parsed['composition'].astype(int).tolist()
        
        m = {
            'scale': (int(d['sd']) - (1-int(mode))) % 8,
            'base': last_comp(d['fb']),
            'sus': last_comp(d['sus']),
            'duration': float(d['chord_duration']),
            'measure': float(d['start_measure']),
            'beat': float(d['start_beat']),
            'beat_abs': float(d['start_beat_abs']),
        }
        
        return cls.from_dict({**parsed, **m})

In [168]:
def default_stream(cls=music21.stream.Score, ts='4/4', bpm=120, ks=0):
    # (AS) TODO: use config ts or metadata
    s = cls()
    s.append(music21.instrument.Piano())
    s.append(music21.meter.TimeSignature(ts))
    s.append(music21.tempo.MetronomeMark(number=bpm))
#     s.append(music21.key.KeySignature(ks))
    s.append(music21.key.Key('C'))
    return s

In [169]:
@dataclass
class HPart(Base):
    notes: List[HNote]
    chords: List[HChord]
        
    @classmethod
    def parse(cls, d, metadata):
        mode = metadata['mode'] or '1'
        key_offset = MODE_TO_KEYOFFSET.get(mode, 0)
        ns = [HNote.parse(n, mode, key_offset) for n in d.get('notes', []) if n['scale_degree'] != 'rest']
        cs = [HChord.parse(c, mode, key_offset) for c in d.get('chords', []) if c['sd'] != 'rest']
        return cls(notes=ns, chords=cs)
    
    
    def duration(self):
        c_last = self.chords[-1].end_time()
        n_last = self.notes[-1].end_time()
        return max(c_last, n_last)
    
    def to_m21(self)->music21.stream.Stream:
        mc = music21.stream.Part()
        mn = music21.stream.Part()
        
        cm21 = [c.to_m21() for c in self.chords]
        for c,d in cm21: mc.insert(d,c)
            
        nm21 = [n.to_m21() for n in self.notes]
        for n,d in nm21: mn.insert(d,n)
        return mn, mc
        
    def min_pitch(self):
        return min([n.pitch for n in self.notes])

In [170]:
@dataclass
class HSong(Base):
    metadata: HMetadata
    parts: List[HPart]
    
    @classmethod
    def parse(cls, metadata, segments):
        m = HMetadata.parse(metadata)
        ps = [HPart.parse(s, metadata) for s in segments]
        return cls(metadata=m, parts=ps)
    
    def duration(self):
        return sum(p.duration() for p in self.parts)
    
    def to_stream(self):
        s = default_stream()
        pc = music21.stream.Part()
        pn = music21.stream.Part()
        
        for p in self.parts:
            mn, mc = p.to_m21()
            pn.append(mn)
            pc.append(mc)
            
        s.insert(0, pn)
        s.insert(0, pc)

#         s.flat.makeNotation(inPlace=True)
        s = s.transpose(0) # hack to get accidentals right. Above method does not work
        # music21 stream
        return s

In [171]:
from src import roman_to_symbol
from src import to_pianoroll
from collections import defaultdict
from midi_data import keyc_offset

In [172]:
import numpy as np
from collections import Counter

In [173]:
all_symbols = []
all_notes = []
all_map = []
jumps = []

In [174]:
s = parse_file(Path('data/midi/v7/midi_sources/hooktheory/xml/w/wyd-krakow-2016/blogoslawieni-milosierni/verse.xml'))

In [191]:
file_path = Path('data/midi/v7/midi_sources/hooktheory/xml/w/wyd-krakow-2016/blogoslawieni-milosierni/verse.xml')
content = load_data(file_path)
root = xml_parser(content)
metadata, version = get_metadata(root)
segments, num_measures = get_lead_sheet(root, version)
    

In [200]:
cdata = segments[-1]['chords'][-2]; cdata

OrderedDict([('sd', '5'),
             ('fb', None),
             ('sec', '4'),
             ('sus', 'sus4'),
             ('pedal', None),
             ('alternate', None),
             ('borrowed', None),
             ('chord_duration', '4'),
             ('start_measure', '3'),
             ('start_beat', '1'),
             ('start_beat_abs', '8'),
             ('isRest', '0')])

In [225]:
def parse_file_emb(file_path):
    content = load_data(file_path)
    root = xml_parser(content)
    metadata, version = get_metadata(root)
    segments, num_measures = get_lead_sheet(root, version)
    for s in segments:
        for c in s['chords']:
            if 'emb' in c: 
                print('Found emb:', version, c['emb'])
                return
            

In [226]:
for f in files[:1000]:
    try: s = parse_file_emb(f)
    except Exception as e: 
        print('Exception:', e)
        continue

Found emb: None None
Found emb: None None
Found emb: None None
Found emb: None None
Found emb: 1.3 None
Found emb: None None
Found emb: 1.3 None
Found emb: None None
Found emb: 1.2 None
Found emb: 1.2 None
Found emb: 1.2 None
Found emb: 1.2 None
Found emb: None add9
Found emb: 1.3 None
Found emb: 1.2 None
Found emb: 1.3 None
Found emb: 1.1 None
Found emb: None add9
Found emb: 1.3 None
Found emb: 1.2 None
Found emb: 1.2 None
Found emb: 1.1 None
Found emb: 1.1 None
Found emb: 1.1 None
Found emb: 1.1 None
Found emb: 1.1 None
Found emb: 1.2 None
Found emb: 1.2 None
Found emb: 1.1 None
Found emb: 1.1 None
Found emb: 1.3 None
Found emb: 1.2 add9
Found emb: 1.3 None
Found emb: 1.1 None
Found emb: None None
Found emb: None add9
Found emb: None add9
Found emb: 1.2 None
Found emb: 1.2 None
Found emb: 1.3 None
Found emb: None None
Found emb: 1.2 add9
Found emb: 1.2 None
Found emb: 1.3 None
Found emb: 1.3 add9
Found emb: 1.3 add9
Found emb: 1.3 None
Found emb: 1.3 None
Found emb: 1.3 None
Found em

In [209]:
parse_file_emb(file_path)

[{'notes': [OrderedDict([('start_beat_abs', '0'),
                ('start_measure', '1'),
                ('start_beat', '1'),
                ('note_length', '2'),
                ('scale_degree', 'rest'),
                ('octave', '0'),
                ('isRest', '1')]),
   OrderedDict([('start_beat_abs', '2'),
                ('start_measure', '1'),
                ('start_beat', '3'),
                ('note_length', '1'),
                ('scale_degree', '1'),
                ('octave', '0'),
                ('isRest', '0')]),
   OrderedDict([('start_beat_abs', '3'),
                ('start_measure', '1'),
                ('start_beat', '4'),
                ('note_length', '0.5'),
                ('scale_degree', '1'),
                ('octave', '0'),
                ('isRest', '0')]),
   OrderedDict([('start_beat_abs', '3.5'),
                ('start_measure', '1'),
                ('start_beat', '4.5'),
                ('note_length', '0.5'),
                ('scale_degree', '2

In [193]:
# cdata['sec'] = '2'

In [194]:
from src.roman_to_symbol import *

In [203]:
KEY_TO_SCALE[MODE_TO_KEY[int('4')]]

[0, 2, 4, 6, 7, 9, 11]

In [ ]:
def set_sus(comp_vec, scale, sd, input_):
    if input_ is None:
        return comp_vec
    elif input_ == 'sus2':
        comp_vec[1] = scale[sd+1]
    elif input_ == 'sus4':
        comp_vec[3] = scale[sd+3]
    elif input_ == 'sus42':
        comp_vec[1] = scale[sd+1]
        comp_vec[3] = scale[sd+3]
    else:
        raise ValueError('Unknown sus: %s' % input_)
    comp_vec[2] = None  # must omit 3
    return comp_vec


In [206]:


def hchord_parser(chord, mode, key_offset):
    if chord['sd'] == 'rest': return None

    # extract basic info
    sd = int(chord['sd']) - 1     # root
    fb = chord['fb']              # tension & inversion
    sec = chord['sec']            # secondary chord
    borrowed = chord.get('borrowed', None)  # borrowed mode

    # determine the mode
    borrowed = is_int(borrowed)
    chord_key = MODE_TO_KEY[int(mode)] if borrowed is None else borrowed
    chord_key = 6 if chord_key > 6 else chord_key
    chord_key = -6 if chord_key < -6 else chord_key

    # secondary chord
    sec_offset = 0
    if sec:
        # switch to 'sec' degree note within the current mode
        scale = KEY_TO_SCALE[MODE_TO_KEY[int(mode)]]
        new_key_note = scale[int(sec) - 1]

        # set that note to new key
        new_key = VAL_TO_NAME[new_key_note][0]

        # get the key shift offset
        sec_offset = NOTE_TO_OFFSET[new_key]
        chord_key = 0

    # determine the scale according to the key(mode) of the chord
    scale = get_scale(chord_key)

    # set compositional notes
    comp, chord_type = set_compositions(scale, fb, sd)

    # determine the quality by triads or seventh
    # (9, 11, 13-th chords are seen as seventh)
    comp_t = comp[0:3] if chord_type is 5 else comp[0:4]
    quality = get_quality(comp_t)

    # add shift from secondary chords
    comp = (comp + sec_offset)
    scale = [s+sec_offset for s in scale]

    # set compvec (for sus/add/omit)
    comp_vec = comp_to_compvec(comp)

    # sus (omit 3)
    sus = chord.get('sus', None)
    comp_vec = set_sus(comp_vec, scale, sd, sus)

    # emb (add/omit)
    if 'emb' in chord:
        emb = chord['emb']
        comp_vec, alter_info, emb_info = set_emb(comp_vec, scale, sd, emb)
    else:
        emb_info = []
        alter_info = []

    # alter (won't change the quality)
    alter_info = alter_info if len(alter_info) else chord.get('alternate', None)
    comp_vec, alter_map = set_alter(comp_vec, alter_info)

    # set inversion (won't change the root, but bass)
    inv = get_num_inversion(fb)
    comp_vec = set_inversion(comp_vec, inv)

    # set result
    comp = compvec_to_comp(comp_vec)
    root = (comp[0] + 120) % 12   # for chord name
    bass = np.nanmin(comp)         # for bass (real root)

    data = OrderedDict([
        # basic compositions
        ('root', root),
        ('bass', bass),
        ('comp_vec', comp_vec),
        ('composition', comp),

        # basic info
        ('quality', quality),
        ('chord_type', chord_type),
        ('chord_mode', chord_key),
        
        # additional info
        ('inv', inv),
        ('sus', sus),
        ('alter', alter_info),
        ('emb', emb_info),
        ('alter_map', alter_map),
        ])

    # key shifting of the symbol
    data = chord_key_shifting(data, key_offset)

    # set chord name
    data['symbol'] = chord_to_string(data)
    return data

In [207]:
hchord_parser(cdata, '6', 0)

OrderedDict([('root', 0),
             ('bass', 12),
             ('comp_vec',
              array([12, None, None, 17, 19, None, None, None, None], dtype=object)),
             ('composition', array([12, 17, 19], dtype=object)),
             ('quality', ''),
             ('chord_type', 5),
             ('chord_mode', 0),
             ('inv', 0),
             ('sus', 'sus4'),
             ('alter', None),
             ('emb', []),
             ('alter_map', None),
             ('symbol', 'C sus4')])

In [176]:
[s['chords'] for s in segments]

[[OrderedDict([('sd', '1'),
               ('fb', '7'),
               ('sec', None),
               ('sus', None),
               ('pedal', None),
               ('alternate', None),
               ('borrowed', None),
               ('chord_duration', '4'),
               ('start_measure', '1'),
               ('start_beat', '1'),
               ('start_beat_abs', '0'),
               ('isRest', '0')]),
  OrderedDict([('sd', '6'),
               ('fb', None),
               ('sec', None),
               ('sus', 'sus2'),
               ('pedal', None),
               ('alternate', None),
               ('borrowed', None),
               ('chord_duration', '4'),
               ('start_measure', '2'),
               ('start_beat', '1'),
               ('start_beat_abs', '4'),
               ('isRest', '0')]),
  OrderedDict([('sd', '3'),
               ('fb', None),
               ('sec', None),
               ('sus', None),
               ('pedal', None),
               ('alternate', Non

In [177]:
[p.chords for p in s.parts]

[[HChord(scale=6, base=7, sus=0, duration=4.0, measure=1.0, beat=1.0, beat_abs=0.0, composition=[0, 3, 7, 10], symbol='cm7', quality='m'),
  HChord(scale=3, base=0, sus=2, duration=4.0, measure=2.0, beat=1.0, beat_abs=4.0, composition=[8, 10, 15], symbol='Ab sus2', quality=''),
  HChord(scale=0, base=0, sus=0, duration=4.0, measure=3.0, beat=1.0, beat_abs=8.0, composition=[3, 7, 10], symbol='Eb', quality=''),
  HChord(scale=7, base=0, sus=0, duration=2.0, measure=4.0, beat=1.0, beat_abs=12.0, composition=[1, 5, 8], symbol='Db', quality=''),
  HChord(scale=3, base=0, sus=2, duration=2.0, measure=4.0, beat=3.0, beat_abs=14.0, composition=[8, 10, 15], symbol='Ab sus2', quality=''),
  HChord(scale=0, base=0, sus=0, duration=4.0, measure=5.0, beat=1.0, beat_abs=16.0, composition=[3, 7, 10], symbol='Eb', quality=''),
  HChord(scale=4, base=0, sus=4, duration=2.0, measure=6.0, beat=1.0, beat_abs=20.0, composition=[10, 15, 17], symbol='Bb sus4', quality=''),
  HChord(scale=4, base=0, sus=0, du

In [178]:
for f in files[:300]:
    try: s = parse_file(f)
    except Exception as e: 
        print('Exception:', e)
        continue
    for p in s.parts:
        for c in p.chords:
            if len(c.composition) > 4: continue
            all_symbols.append(c.symbol)
            chords = list(([i%12 for i in sorted(c.composition)]))
            all_notes.append(str(chords))
            all_map.append(c.symbol + ' - ' + str(chords))
            jumps.append(str(np.diff(sorted(c.composition))))
            j = np.diff(c.composition)
#             if np.any(j > 6): print(c.symbol)
            if np.any(j == 0): print(c.symbol, c.composition, f)
#             print(c.symbol, c.composition, j)

C sus4 [12, 12, 19] data/midi/v7/midi_sources/hooktheory/xml/w/wolfgang-amadeus-mozart/kyrie-from-mass-in-c-minor-k-427/verse.xml
C sus4 [12, 12, 19] data/midi/v7/midi_sources/hooktheory/xml/w/wolfgang-amadeus-mozart/kyrie-from-mass-in-c-minor-k-427/verse.xml
C7 sus4 [12, 12, 19, 22] data/midi/v7/midi_sources/hooktheory/xml/w/wolfgang-amadeus-mozart/confutatis-from-requiem/bridge.xml
C sus4 [12, 12, 19] data/midi/v7/midi_sources/hooktheory/xml/w/wyd-krakow-2016/blogoslawieni-milosierni/verse.xml
C sus4 [12, 12, 19] data/midi/v7/midi_sources/hooktheory/xml/w/wyd-krakow-2016/blogoslawieni-milosierni/verse.xml


In [137]:
jumps_count = Counter(jumps); len(jumps_count)

63

In [138]:
jumps_count.most_common()

[('[4 3]', 7643),
 ('[3 4]', 3650),
 ('[3 4 3]', 975),
 ('[3 5]', 713),
 ('[4 3 4]', 647),
 ('[4 3 3]', 637),
 ('[5 4]', 457),
 ('[4 5]', 303),
 ('[3 3]', 222),
 ('[5 3]', 198),
 ('[5 2]', 186),
 ('[4 3 7]', 161),
 ('[4 3 2]', 118),
 ('[2 3 4]', 105),
 ('[3 3 2]', 99),
 ('[2 4 3]', 98),
 ('[5 2 3]', 88),
 ('[3 3 4]', 86),
 ('[2 5]', 85),
 ('[3 4 7]', 75),
 ('[3 2 4]', 61),
 ('[3 4 2]', 49),
 ('[2 5 3]', 36),
 ('[3 4 1]', 32),
 ('[1 4 3]', 31),
 ('[2 3 3]', 29),
 ('[3 2 3]', 27),
 ('[3 6]', 25),
 ('[6 3]', 24),
 ('[2 5 4]', 23),
 ('[2 3 2]', 20),
 ('[3 5 2]', 17),
 ('[4 1 4]', 12),
 ('[5 2 2]', 12),
 ('[4 2 3]', 11),
 ('[0 7]', 10),
 ('[6 1]', 6),
 ('[0 7 3]', 6),
 ('[4 5 2]', 5),
 ('[2 5 2]', 4),
 ('[2 7]', 4),
 ('[1 6 3]', 4),
 ('[5 7 3]', 3),
 ('[5 2 4]', 3),
 ('[1 5 4]', 3),
 ('[4 5 1]', 3),
 ('[2 7 3]', 2),
 ('[4 7 3]', 2),
 ('[3 2 5]', 2),
 ('[5 5]', 2),
 ('[3 0 3]', 1),
 ('[5 7]', 1),
 ('[5 1]', 1),
 ('[1 5]', 1),
 ('[6 1 4]', 1),
 ('[4 7]', 1),
 ('[4 4 3]', 1),
 ('[5 1 2]', 1),


In [139]:
# quality - 

In [140]:
symbol_count = Counter(all_symbols); len(symbol_count)

299

In [141]:
symbol_count.most_common()

[('C', 2396),
 ('F', 2085),
 ('G', 2049),
 ('am', 2004),
 ('dm', 751),
 ('em', 637),
 ('E', 331),
 ('Fmaj7', 328),
 ('dm7', 315),
 ('am7', 297),
 ('Bb', 275),
 ('C | E', 254),
 ('em7', 248),
 ('G7', 241),
 ('G | B', 200),
 ('C | G', 158),
 ('Cmaj7', 138),
 ('F | C', 134),
 ('D', 130),
 ('G | D', 122),
 ('E7', 119),
 ('A', 114),
 ('Ab', 104),
 ('am | E', 96),
 ('am | C', 91),
 ('F | A', 90),
 ('em | G', 90),
 ('Eb', 83),
 ('D7', 78),
 ('Bbmaj7', 74),
 ('A7', 73),
 ('dm | F', 70),
 ('G sus4', 68),
 ('fm', 67),
 ('E | Ab', 65),
 ('C7', 62),
 ('abo', 61),
 ('am7 | G', 60),
 ('gm', 59),
 ('cm', 57),
 ('F (add9)', 55),
 ('dm7 | F', 52),
 ('em | B', 52),
 ('gbo', 47),
 ('D | Gb', 47),
 ('bo', 44),
 ('B', 44),
 ('dm (add9)', 43),
 ('G7 | F', 43),
 ('bm', 42),
 ('C (add9)', 42),
 ('G (add9)', 41),
 ('gbø7', 38),
 ('C sus4', 37),
 ('Abmaj7', 36),
 ('am7 | C', 35),
 ('G7 sus4', 30),
 ('dm | A', 30),
 ('bm7', 29),
 ('Gmaj7', 29),
 ('dm7 | C', 29),
 ('bø7', 27),
 ('gm7', 27),
 ('E7 | Ab', 26),
 ('G

In [142]:
note_count = Counter(all_notes); len(note_count)

294

In [143]:
note_count.most_common()

[('[0, 4, 7]', 2396),
 ('[5, 9, 0]', 2085),
 ('[7, 11, 2]', 2049),
 ('[9, 0, 4]', 2008),
 ('[2, 5, 9]', 751),
 ('[4, 7, 11]', 637),
 ('[4, 8, 11]', 331),
 ('[5, 9, 0, 4]', 328),
 ('[2, 5, 9, 0]', 315),
 ('[9, 0, 4, 7]', 298),
 ('[10, 2, 5]', 275),
 ('[4, 7, 0]', 254),
 ('[4, 7, 11, 2]', 248),
 ('[7, 11, 2, 5]', 241),
 ('[11, 2, 7]', 200),
 ('[7, 0, 4]', 158),
 ('[0, 4, 7, 11]', 138),
 ('[0, 5, 9]', 134),
 ('[2, 6, 9]', 130),
 ('[2, 7, 11]', 122),
 ('[4, 8, 11, 2]', 119),
 ('[9, 1, 4]', 114),
 ('[8, 0, 3]', 104),
 ('[4, 9, 0]', 96),
 ('[0, 4, 9]', 91),
 ('[9, 0, 5]', 90),
 ('[7, 11, 4]', 90),
 ('[3, 7, 10]', 83),
 ('[2, 6, 9, 0]', 78),
 ('[10, 2, 5, 9]', 74),
 ('[9, 1, 4, 7]', 73),
 ('[5, 9, 2]', 70),
 ('[7, 0, 2]', 68),
 ('[5, 8, 0]', 67),
 ('[8, 11, 4]', 65),
 ('[0, 4, 7, 10]', 62),
 ('[7, 9, 0, 4]', 61),
 ('[8, 11, 2]', 61),
 ('[7, 10, 2]', 59),
 ('[0, 3, 7]', 57),
 ('[5, 9, 0, 7]', 55),
 ('[5, 9, 0, 2]', 52),
 ('[11, 4, 7]', 52),
 ('[6, 9, 0]', 47),
 ('[6, 9, 2]', 47),
 ('[11, 2, 5]

chord without inversion

In [ ]:


# def hchord_parser(chord, mode, key_offset):
#     if chord['sd'] == 'rest': return None

#     # extract basic info
#     sd = int(chord['sd']) - 1     # root
#     fb = chord['fb']              # tension & inversion
#     sec = chord['sec']            # secondary chord
#     borrowed = chord.get('borrowed', None)  # borrowed mode

#     # determine the mode
#     borrowed = is_int(borrowed)
#     chord_key = MODE_TO_KEY[int(mode)] if borrowed is None else borrowed
#     chord_key = 6 if chord_key > 6 else chord_key
#     chord_key = -6 if chord_key < -6 else chord_key

#     # secondary chord
#     sec_offset = 0
#     if sec:
#         # switch to 'sec' degree note within the current mode
#         scale = KEY_TO_SCALE[MODE_TO_KEY[int(mode)]]
#         new_key_note = scale[int(sec) - 1]

#         # set that note to new key
#         new_key = VAL_TO_NAME[new_key_note][0]

#         # get the key shift offset
#         sec_offset = NOTE_TO_OFFSET[new_key]
#         chord_key = 0

#     # determine the scale according to the key(mode) of the chord
#     scale = get_scale(chord_key)

#     # set compositional notes
#     comp, chord_type = set_compositions(scale, fb, sd)

#     # determine the quality by triads or seventh
#     # (9, 11, 13-th chords are seen as seventh)
#     comp_t = comp[0:3] if chord_type is 5 else comp[0:4]
#     quality = get_quality(comp_t)

#     # add shift from secondary chords
#     comp = (comp + sec_offset)

#     # set compvec (for sus/add/omit)
#     comp_vec = comp_to_compvec(comp)

#     # sus (omit 3)
#     sus = chord.get('sus', None)
#     comp_vec = set_sus(comp_vec, scale, sd, sus)

#     # emb (add/omit)
#     if 'emb' in chord:
#         emb = chord['emb']
#         comp_vec, alter_info, emb_info = set_emb(comp_vec, scale, sd, emb)
#     else:
#         emb_info = []
#         alter_info = []

#     # alter (won't change the quality)
#     alter_info = alter_info if len(alter_info) else chord.get('alternate']
#     comp_vec, alter_map = set_alter(comp_vec, alter_info)

#     # set inversion (won't change the root, but bass)
#     inv = get_num_inversion(fb)
#     comp_vec = set_inversion(comp_vec, inv)

#     # set result
#     comp = compvec_to_comp(comp_vec)
#     root = (comp[0] + 120) % 12   # for chord name
#     bass = np.nanmin(comp)         # for bass (real root)

#     data = OrderedDict([
#         # basic compositions
#         ('root', root),
#         ('bass', bass),
#         ('comp_vec', comp_vec),
#         ('composition', comp),

#         # basic info
#         ('quality', quality),
#         ('chord_type', chord_type),
#         ('chord_mode', chord_key),
        
#         # additional info
#         ('inv', inv),
#         ('sus', sus),
#         ('alter', alter_info),
#         ('emb', emb_info),
#         ('alter_map', alter_map),
#         ])

#     # key shifting of the symbol
#     data = chord_key_shifting(data, key_offset)

#     # set chord name
#     data['symbol'] = chord_to_string(data)
#     return data

In [180]:
VAL_TO_NAME = {
    1: 
}

In [ ]:
def chord_to_string(data):
    base = to_name(data['root']) + data['quality']
    if data['quality'] in ['m', 'ø', 'o']:
        base = base.lower()
    base += ('' if data['chord_type'] is 5 else str(data['chord_type']))
    if data['inv']:
        base += (' | ' + to_name(data['bass']))
    if data['sus'] is not None:
        base += (' ' + data['sus'])
    if data['emb'] is not None:
        for e in data['emb']:
            base += ' (%s)' % e
    if data['alter'] is not None:
        for a in data['alter']:
            base += ' (%s)' % a
    return base